In [2]:
import requests
# import json
import pandas as pd
# import boto3
# import numpy as np
# import time
from bs4 import BeautifulSoup
# from botocore.exceptions import ClientError
import unicodedata
# import lxml
# import base64

In [6]:
api_key = '627339a4-1f8a-4fc8-923e-9bf4dd73fd46'
payload = {
"username": 'clinops_ai_svc_acct@gilead.com',
"password": 'Os@DFEztB9zq',
"privateKey": api_key
}

url = f'https://rest.myabsorb.com/authenticate'

resp = requests.post(url, json = payload, headers={'x-api-key':api_key})
token = resp.json()

#Get all course list
courses = []
page_num = 0
flag = True
while flag==True:
    url = f"https://rest.myabsorb.com/courses?_limit=1000&_offset={page_num}"
    payload = {}
    headers = {
        'Authorization': token,
        'x-api-key': api_key,
        'Content-Type': 'application/json'
    }
    page_num +=1
    resp = requests.request("GET", url, headers=headers, data=payload)
    courses.extend(resp.json())
    print(resp,bool(resp.json()),page_num,len(resp.json()))
    if bool(resp.json())==False:
        flag=False

print(len(courses))
courses = courses[:-1]

#get filtered course list
not_req = ['GxPLearn','G.Learn','LinkedIn Learning']
coursesdf = pd.DataFrame(data=courses)
df = coursesdf[(coursesdf.ActiveStatus==0)&(coursesdf['ExternalId'].str.strip().str.contains("G.Learn",case=False,na=False))&(~(coursesdf.Name.str.strip().str.contains("instructor-led course",case=False)))].reset_index(drop = True)

def category(row):
    id = row['CategoryId']
    try:
        url = f'https://rest.myabsorb.com/categories/{id}'
        cat_id = requests.get(url, headers={'x-api-key':api_key,'Authorization':token})
        return cat_id.json()['Name']
    except:
        return None

df['Category'] = df.apply(category, axis=1)

mapping = {'OnlineCourse':'online-courses', 'InstructorLedCourse':'instructor-led-courses','Curriculum':'Curriculum'}
df['coursetype_new'] = df['CourseType'].map(mapping)
df['deeplink'] = 'https://gilead.myabsorb.com/#/' + df['coursetype_new'] + '/'+ df['Id']
df.drop('coursetype_new',axis=1,inplace=True)

df.fillna('None',inplace=True)
def formatdesc(row):
    try:
        desc = BeautifulSoup(row['Description'], 'lxml').text  #removes html tag
        desc = ' '.join([line.strip() for line in desc.strip().splitlines() if line.strip()]) #remove new line character
        desc = unicodedata.normalize('NFKC', desc) #to remove unicode characters
        return desc
    except:
        return None

df["Description"]=df.apply(formatdesc, axis=1)
glearn_df = df.reset_index(inplace=True)

print(len(glearn_df))
glearn_df.head()

<Response [200]> True 1 1000
<Response [200]> True 2 246
<Response [200]> False 3 0
1246
31


,index,Id,CourseType,Name,Description,Notes,ExternalId,AccessDate,ExpireType,ExpireDuration,...,Goals,Vendor,CompanyCost,LearnerCost,CompanyTime,LearnerTime,DateEdited,DateAdded,Category,deeplink
0,0,aef2f5fb-0add-4143-b4ec-c9156ad94fad,OnlineCourse,Adverse Effects of Antiretroviral Agents (G.Le...,This module provides an overview of the curren...,None,G.Learn 1359334,None,0,"{'Years': 0, 'Months': 0, 'Days': 0, 'Hours': 0}",...,None,None,None,None,None,None,2023-03-10T17:02:08.2,2018-09-19T17:44:09.703,HIV Treatment,https://gilead.myabsorb.com/#/online-courses/a...
1,1,3865714b-dde8-425b-a58a-9815599b1bec,OnlineCourse,Antiretroviral Agents (G.Learn Course 1359331),This module provides an overview of the curren...,None,G.Learn 1359331,None,0,"{'Years': 0, 'Months': 0, 'Days': 0, 'Hours': 0}",...,None,None,None,None,None,None,2023-03-10T16:58:30.05,2018-09-19T17:04:06.637,HIV Treatment,https://gilead.myabsorb.com/#/online-courses/3...
2,2,68c8a37b-23ee-4ce2-8e6f-053dc909879d,OnlineCourse,Antiretroviral Therapy Toxicity (G.Learn Cours...,In this Medical Affairs Lecture Series present...,None,G.Learn 1368330,None,0,"{'Years': 0, 'Months': 0, 'Days': 0, 'Hours': 0}",...,None,None,None,None,None,None,2023-03-10T17:03:30.92,2018-09-19T17:57:56.447,HIV Treatment,https://gilead.myabsorb.com/#/online-courses/6...
3,3,b002c729-7d26-41d1-8c75-50a70d84c879,OnlineCourse,AOEM_WW_HIV_Training Bulletin – Understanding ...,This module provides an overview of laboratory...,Purpose: To learn about HIV related Laboratory...,G.Learn 1404330,None,0,"{'Years': 0, 'Months': 0, 'Days': 0, 'Hours': 0}",...,None,Suzi Saroukhanians / Jami Petrie,None,None,None,None,2023-05-25T16:54:50.887,2022-03-18T15:41:01.473,HIV Basics,https://gilead.myabsorb.com/#/online-courses/b...
4,4,74cd1ede-0e03-45d7-8523-f84f41d4ec43,OnlineCourse,ART Issues with Adherence and Resistance (G.Le...,This module provides an overview of how adhere...,None,G.Learn 1368329,None,0,"{'Years': 0, 'Months': 0, 'Days': 0, 'Hours': 0}",...,None,None,None,None,None,None,2023-03-10T16:59:24.77,2018-09-19T17:30:42.817,HIV Treatment,https://gilead.myabsorb.com/#/online-courses/7...
5,5,00118781-c679-4ea7-993c-298d1fbecb6f,OnlineCourse,Diagnosis of HBV Infection (G.Learn Course 143...,This module provides an overview of the steps ...,None,G.Learn 1433328,None,0,"{'Years': 0, 'Months': 0, 'Days': 0, 'Hours': 0}",...,None,None,None,None,None,None,2023-03-10T16:52:57.767,2018-09-19T00:23:34.433,Hepatitis B Virus (HBV),https://gilead.myabsorb.com/#/online-courses/0...
6,6,175a8a34-eeef-49dc-b3ba-f18dcd64000f,OnlineCourse,Diagnosis of HCV Infection (G.Learn Course 128...,The course begins with an overview of the clin...,None,G.Learn 1433328,None,0,"{'Years': 0, 'Months': 0, 'Days': 0, 'Hours': 0}",...,None,None,None,None,None,None,2023-03-10T16:54:57.69,2018-09-19T16:01:08.16,Hepatitis C Virus (HCV),https://gilead.myabsorb.com/#/online-courses/1...
7,7,14272123-a024-487f-822d-62d5d33be370,OnlineCourse,Diagnosis of HIV Infection and AIDS (G.Learn C...,This module provides an overview of the proces...,None,G.Learn 1359330,None,0,"{'Years': 0, 'Months': 0, 'Days': 0, 'Hours': 0}",...,None,None,None,None,None,None,2023-03-10T16:57:46.76,2018-09-19T16:58:09.9,HIV Basics,https://gilead.myabsorb.com/#/online-courses/1...
8,8,fee4752d-3743-48a4-90c5-2295e803b815,OnlineCourse,Gilead Product Overviews (G.Learn Course 1878353),High level information on Gilead products from...,None,G.Learn 1878353,None,0,"{'Years': 0, 'Months': 0, 'Days': 0, 'Hours': 0}",...,None,None,None,None,None,None,2023-03-10T17:11:28.38,2021-12-13T17:59:54.523,Gilead Onboarding,https://gilead.myabsorb.com/#/online-courses/f...
9,9,a3c2d24d-d260-4dfc-889f-d6ba4dbc3b3c,OnlineCourse,"HBV - Liver Anatomy, Physiology, and Histology...",This module provides an overview of the liver ...,None,G.Learn 1433326,None,0,"{'Years': 0, 'Months': 0, 'Days': 0, 'Hours': 0}",...,None,None,None,None,None,None,2023-03-10T16:50:07.837,2018-09-14T21:27:52.807